In [4]:
import numpy as np
import scipy.spatial.distance
from imutils.object_detection import non_max_suppression
import scipy.signal
from skimage.segmentation import slic
from skimage.util import img_as_float
from scipy.optimize import minimize

import os
import scipy.io
import cv2
import math

from skimage import io
import itertools

In [5]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [6]:
File_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Dataset/Location_Appended_images/Output/11Channel/final_featureMap/5FeatureMap/HS_salmap (copy)'
HS_files = sorted(os.listdir(File_path))
print("HS_files", HS_files)
# num = 0;
save_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach8/Output_Image_Dataset/Full_System_Output'
# save_path_full_system_nms = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach5/Output_Image_Dataset/Location_Appended_Images/Full_System_NMS'

HS_files ['0001.png', '0002.png', '0006.png', '0008.png', '0009.png', '0010.png', '0012.png', '0014.png', '0015.png', '0016.png', '0017.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0024.png', '0026.png', '0027.png', '0028.png', '0029.png', '0031.png', '0032.png', '0033.png', '0034.png', '0036.png', '0037.png', '0038.png', '0039.png', '0040.png', '0041.png', '0042.png', '0043.png', '0044.png', '0045.png', '0046.png', '0047.png', '0050.png', '0051.png', '0052.png', '0053.png', '0054.png', '0056.png', '0058.png', '0061.png', '0062.png', '0063.png', '0064.png', '0065.png', '0068.png', '0069.png', '0070.png', '0071.png', '0072.png', '0074.png', '0076.png', '0077.png', '0078.png', '0079.png', '0082.png']


In [11]:
for file in HS_files:
# for file in file_list:

    # Code for Erosion
    input_img = os.path.join(File_path, file)
    saliency_map = cv2.imread(input_img, cv2.IMREAD_GRAYSCALE)
    # print('saliency_map shape', saliency_map.shape)
    file_name = file.rsplit('.', 1)
    numpydata_in = np.array(saliency_map)
    # numpydata_in = np.mean(numpydata_in, axis = 2)
    numpydata_in = numpydata_in.astype('uint8')
    #numpydata_in = cv2.medianBlur(numpydata_in,)
    kernel = np.ones((1,1),np.uint8)
    saliency_map = cv2.erode(numpydata_in,kernel,iterations=1)


    # Code for NMS

    # Threshold the saliency map to obtain a binary mask
    threshold = 128  # adjust this threshold as needed
    mask = (saliency_map > threshold).astype(np.uint8)

    # Find the connected components in the binary mask
    components, labels = cv2.connectedComponents(mask)

    # Initialize a list to store the bounding boxes and scores of each component
    boxes = []
    scores = []

    # Loop over each component and compute its bounding box and score
    for i in range(1, np.amax(components) + 1):
        # Extract the binary mask for the current component
        component_mask = (labels == i).astype(np.uint8)

        # Compute the bounding box of the component
        bbox = cv2.boundingRect(component_mask)

        # Compute the score of the component as the maximum saliency value within the component
        if np.count_nonzero(component_mask) > 0:
            score = np.max(saliency_map[component_mask == 1])
        else:
            score = 0

        # Append the bounding box and score to the list
        boxes.append(bbox)
        scores.append(score)

    # Apply non-maximum suppression to the list of bounding boxes and scores
    overlap_thresh = 0.5  # adjust this threshold as needed
    scores = np.array(scores)
    boxes = np.array(boxes)
    selected_indices = []

    # Flatten the scores array before performing argsort
    order = scores.ravel().argsort()[::-1]
    areas = (boxes[:, 2] * boxes[:, 3])

    if np.any(areas == 0):
        areas[areas == 0] = 1

    # Loop over the indices in the order determined by the scores and compute the overlap of each bounding box with the previously selected bounding boxes
    while order.size > 0:
        # Select the bounding box with the highest score and add it to the list of selected boxes
        i = order[0]
        selected_indices.append(i)

        # Compute the overlap of the selected bounding box with the other bounding boxes
        xx1 = np.maximum(boxes[i, 0], boxes[order[1:], 0])
        yy1 = np.maximum(boxes[i, 1], boxes[order[1:], 1])
        xx2 = np.minimum(boxes[i, 0] + boxes[i, 2], boxes[order[1:], 0] + boxes[order[1:], 2])
        yy2 = np.minimum(boxes[i, 1] + boxes[i, 3], boxes[order[1:], 1] + boxes[order[1:], 3])
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[order[1:]]

        # Remove bounding boxes with overlap greater than the overlap threshold
        inds = np.where(overlap <= overlap_thresh)[0]
        order = order[inds + 1]

    # Loop over the selected bounding boxes and draw them on the image
    img_rgb = cv2.cvtColor(saliency_map, cv2.COLOR_GRAY2RGB)
    for i in selected_indices:
        x, y, w, h = boxes[i]
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 0, 0), 0)

    # save_im_final_featureMap = save_path_full_system_nms + '/' + file_name[0] + '.png'
    # cv2.imwrite(save_im_final_featureMap, img_rgb)

    # Code for Top-Five

    area = []
    topFiveBoxes = []
    i = 0
    for dim in boxes:
        _, _, length, width = dim
        area.append((i, length * width))
        i += 1

    # print('area', area)
    sorted_list = sorted(area, key=lambda x: x[1], reverse=True)
    # print(sorted_list)
    exceptFive = sorted_list[5:]
    # print('exceptFive', exceptFive)
    topFive = sorted_list[:5]
    # print('topFive', topFive)

    exceptFiveBoxes = []
    for item in exceptFive:
        exceptFiveBoxes.append(boxes[item[0]])

    topFiveBoxes = []
    for item in topFive:
        # print(1)
        topFiveBoxes.append(boxes[item[0]])
    # print('topFiveBoxes', len(topFiveBoxes))

    for box in exceptFiveBoxes:
        x, y, w, h = box
        if w > 0 and h > 0: 
            area = img_rgb[y:y+h, x:x+w]
            gray_roi = np.zeros((h, w), dtype=np.uint8)
            img_rgb[y:y+h, x:x+w] = cv2.cvtColor(gray_roi, cv2.COLOR_GRAY2BGR)


    # Code For Boundary Pixel

    ratio_data = []

    for box in topFiveBoxes:
        x, y, w, h = box
        if w > 0 and h > 0:
            area_dim = img_rgb[y:y+h, x:x+w]

            length = area_dim.shape[0]
            # print('length', length)
            width = area_dim.shape[1]
            # print('width', width)

            boundary_pixel_cnt = 0
            for i in range(length):
                for j in range(width):
                    if i == 0 or i == (length - 1):
                        boundary_pixel_cnt += 1
                    if j == 0 or j == (width - 1):
                        boundary_pixel_cnt += 1
            
            ratio_data.append([boundary_pixel_cnt, length * width])
        # print('boundary pixel count', boundary_pixel_cnt)
    # print('boundary pixel ratio', ratio_data)

    boundary_pixel_ratio = []
    for item in ratio_data:
        val = item[1]/(1 + item[0])
        boundary_pixel_ratio.append(val)
    # print('boundary pixel ratio', boundary_pixel_ratio)

    max = np.max(boundary_pixel_ratio)
    index_of_max = boundary_pixel_ratio.index(max)
    # print('index of max', index_of_max)

    for i in range(len(topFiveBoxes)):
        if i == index_of_max:
            # print('i in if', i)
            continue
        else:
            # print('i in else', i)
            x, y, w, h = topFiveBoxes[i]
            if w > 0 and h > 0: 
                area = img_rgb[y:y+h, x:x+w]
                gray_roi = np.zeros((h, w), dtype=np.uint8)
                # white_roi = 255 - gray_roi
                img_rgb[y:y+h, x:x+w] = cv2.cvtColor(gray_roi, cv2.COLOR_GRAY2BGR)

    save_im_final_featureMap = save_path + '/' + file_name[0] + '.jpg'
    cv2.imwrite(save_im_final_featureMap, img_rgb)